In [ ]:
import os
import cv2
import numpy as np

# Define the paths
dataset_path = r"C:\Users\adila\OneDrive\Desktop\Project\Datasets\Curated Dataset for COVID-19 Posterior-Anterior Chest Radiography Images (X-Rays)\Curated Dataset for COVID-19 Posterior-Anterior Chest Radiography Images (X-Rays)\Curated X-Ray Dataset\Curated X-Ray Dataset"
equalized_path = r"C:\Users\adila\OneDrive\Desktop\adaptive CLAHE, non-local means denoising, and power law"

# Function to create directory structure similar to the Dataset folder structure
def create_equalised_structure(input_path, output_path):
    for root, dirs, files in os.walk(input_path):
        for dir in dirs:
            output_subdir_path = os.path.join(output_path, os.path.relpath(os.path.join(root, dir), input_path))
            os.makedirs(output_subdir_path, exist_ok=True)

# Create the output directory structure within the EQUALISED folder
create_equalised_structure(dataset_path, equalized_path)

def process_images(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".jpg"):
                input_image_path = os.path.join(root, file)
                output_subdir = root.replace(input_dir, output_dir)
                os.makedirs(output_subdir, exist_ok=True)  # Ensure the output subdirectory exists
                output_image_path = os.path.join(output_subdir, file)
                
                # Read the image
                image = cv2.imread(input_image_path)
                
                # Convert image to grayscale
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # Apply adaptive CLAHE
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                clahe_image = clahe.apply(gray_image)

                # Apply non-local means denoising
                denoised_image = cv2.fastNlMeansDenoising(clahe_image, None, h=10, templateWindowSize=7, searchWindowSize=21)
                
                # Apply power law transformation with gamma 1.5
                gamma = 1.5
                power_law_image = np.power(denoised_image / 255.0, gamma)
                power_law_image = np.uint8(power_law_image * 255)
                
                # Save the processed image
                cv2.imwrite(output_image_path, power_law_image)

# Process images in the Dataset directory
process_images(dataset_path, equalized_path)

print("Grayscale conversion, adaptive CLAHE, non-local means denoising, and power law transformation completed.")
